In [1]:
import numpy as np
import random
from wordPreprocess import WordPreprocess



In [2]:
program1 = np.load('ex_Program1.npy')
questions = np.load('../data/cut_Questions.npy')
voc_dict = np.load('../data/voc_dict.npy')

In [3]:
word_preprocess = WordPreprocess()

program1 = word_preprocess.question_parse(questions=questions, program=program1, word_dict=voc_dict)

In [6]:
doc_hash = word_preprocess.hash_transform(program1, num_feature=2**9)

In [13]:
program = doc_hash[:227075]
questions = doc_hash[227075:]

In [15]:
NUM_TRAIN = 20000
def generate_training_data():
    Xs, Ys = [], []
    
    for i in range(NUM_TRAIN):
        pos_or_neg = random.randint(0,1)
        
        if pos_or_neg is 1:
            line_id = random.randint(0, program.shape[0]-1)
            
            Xs.append([program[line_id], program[line_id+1]])
            Ys.append(1)
            
        else:
            line1_id = random.randint(0, program.shape[0]-1)
            line2_id = random.randint(0, program.shape[0]-1)
            
            Xs.append([program[line1_id], program[line2_id]])
            Ys.append(0)
            
    return Xs, Ys

In [16]:
from sklearn.model_selection import train_test_split

X, y = generate_training_data()

X_ = []

for t in X:
    X_.append([x+y for x, y in zip(t[0], t[1])])


X_ = np.stack(X_, axis=0)
print (X_)

X_train, X_valid, y_train, y_valid = train_test_split(X_, y, test_size=0.3, random_state=1)

[[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]


In [17]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_valid_std = sc.transform(X_valid)

In [18]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

svm_rbf = SVC(kernel='rbf', random_state=0, C=10.0, gamma=0.2)
svm_rbf.fit(X_train_std, y_train)

y_pred = svm_rbf.predict(X_valid_std)

print ("Accuracy: %.3f" % (accuracy_score(y_valid, y_pred)))

Accuracy: 0.512


In [19]:
q_test = []
for q_id in range(500):
    question = questions[q_id*7]
    for i in range(1,7):
        ans = questions[q_id*7 + i]
        set_ = [x+y for x, y in zip(question, ans)]
        set_ = np.array(set_)
        result = svm_rbf.predict(set_.reshape(1,-1))
        if result == 1:
            q_test.append(i)
            break
        elif (i == 6 and result == 0):
            q_test.append(random.randint(1,6))